In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 waste_cnt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)  
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110620

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,109006.1970
2018-02-28,77626.2063
2018-03-31,84364.3619
2018-04-30,66239.5073
2018-05-31,57599.3845
2018-06-30,59147.1368
2018-07-31,84671.0775
2018-08-31,103098.1231
2018-09-30,92804.5874


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31    109006.1970
 2018-02-28     77626.2063
 2018-03-31     84364.3619
 2018-04-30     66239.5073
 2018-05-31     57599.3845
 2018-06-30     59147.1368
 2018-07-31     84671.0775
 2018-08-31    103098.1231
 2018-09-30     92804.5874
 2018-10-31     66594.2574
 2018-11-30     78926.6088
 2018-12-31     88899.7050
 2019-01-31    122252.1178
 2019-02-28     96404.7040
 2019-03-31    127894.5430
 2019-04-30     86262.1903
 2019-05-31     73076.7596
 2019-06-30     87439.8247
 2019-07-31     88455.8214
 2019-08-31    164821.1678
 2019-09-30     90221.4962
 2019-10-31     85552.7348
 2019-11-30    105149.0269
 2019-12-31    109657.5280
 2020-01-31     97153.3374
 2020-02-29    100008.7081
 2020-03-31    125808.9303
 2020-04-30     81691.6753
 2020-05-31     69132.5673
 2020-06-30     79382.9854
 2020-07-31     91105.8813
 2020-08-31    134819.5562
 2020-09-30     69720.8742
 2020-10-31     67836.0336
 2020-11-30     71660.9480
 2020-12-31    103726.8115

# ARIMA

In [6]:
# ARIMA에 적합한 log로 바꾸는 과정
sub_df_log = np.log1p(sub_df[:])
#sub_df_float.tail()

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df_log)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -2.028685
p-value : 0.274139
Critical Values :
	1%: -3.653520
	5%: -2.957219
	10%: -2.617588


In [8]:
# 차분
diff_1 = sub_df_log.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -2.486529
p-value : 0.118786
Critical Values :
	1%: -3.669920
	5%: -2.964071
	10%: -2.621171


In [9]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df_log,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df_log)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=1.551, Time=0.03 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=-0.224, Time=0.12 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=1.626, Time=0.08 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=1.452, Time=0.13 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=2.736, Time=0.15 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=inf, Time=0.49 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=0.066, Time=0.04 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=1.758, Time=0.10 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=3.770, Time=0.15 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=3.801, Time=0.36 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=1.708, Time=0.41 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=1.455, Time=0.14 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=3.482, Time=0.24 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=5.535, Time=0.32 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=6.303, Time=0.41 sec
 ARIMA(3,0,0)(0,0,0)[1] intercept   : AIC=3.294, Time=0.

ARIMA(order=(0, 0, 1), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [10]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df_log.long_term_frgn.values, order=(0,0,1))    

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(0, 1)   Log Likelihood                   3.112
Method:                       css-mle   S.D. of innovations              0.224
Date:                Sat, 11 Sep 2021   AIC                             -0.224
Time:                        04:14:37   BIC                              4.989
Sample:                             0   HQIC                             1.687
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         11.3683      0.045    253.427      0.000      11.280      11.456
ma.L1.y        0.3025      0.141      2.142      0.032       0.026       0.579
                                    Roots           

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [11]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([11.26672833, 11.36826418]), array([0.22443289, 0.23447454]), array([[10.82684796, 11.70660871],
       [10.90870251, 11.82782584]]))


In [12]:
print( np.exp(11.26672833) - 1 )   
print( np.exp(11.36826418) - 1 )  

78175.80959035543
86530.5338789192
